In [ ]:
%matplotlib inline

In [ ]:
# This part is from https://mne.tools/stable/auto_tutorials/machine-learning/plot_sensors_decoding.html 

import numpy as np
import matplotlib.pyplot as plt

from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LogisticRegression

import mne
from mne.datasets import sample
from mne.decoding import (SlidingEstimator, GeneralizingEstimator, Scaler,
                          cross_val_multiscore, LinearModel, get_coef,
                          Vectorizer, CSP)

In [ ]:
data_path = sample.data_path()

subjects_dir = data_path + '/subjects'
raw_fname = data_path + '/MEG/sample/sample_audvis_raw.fif'
tmin, tmax = -0.200, 0.500
event_id = {'Auditory/Left': 1, 'Visual/Left': 3}  # just use two
raw = mne.io.read_raw_fif(raw_fname, preload=True)

# The subsequent decoding analyses only capture evoked responses, so we can
# low-pass the MEG data. Usually a value more like 40 Hz would be used,
# but here low-pass at 20 so we can more heavily decimate, and allow
# the examlpe to run faster. The 2 Hz high-pass helps improve CSP.
raw.filter(2, 20)
events = mne.find_events(raw, 'STI 014')

# Set up pick list: EEG + MEG - bad channels (modify to your needs)
raw.info['bads'] += ['MEG 2443', 'EEG 053']  # bads + 2 more

# Read epochs
epochs = mne.Epochs(raw, events, event_id, tmin, tmax, proj=True,
                    picks=('eeg', 'eog'), baseline=(None, 0.), preload=True,
                    reject=dict(eeg=150e-6, eog=150e-6), decim=10)
epochs.pick_types(meg=False, eeg=True, exclude='bads')  # remove stim and EOG
del raw

X = epochs.get_data()  # MEG signals: n_epochs, n_meg_channels, n_times
y = epochs.events[:, 2]  # target: Audio left or right

In [ ]:
epochs.info

In [ ]:
y

In [ ]:
y[y==3] = 0

In [ ]:
y

In [ ]:
from sklearn.model_selection import StratifiedShuffleSplit

In [ ]:
cv = StratifiedShuffleSplit(n_splits=5, test_size=0.2, random_state=124)

In [ ]:
clf = make_pipeline(StandardScaler(), LogisticRegression())

In [ ]:
time_decod = SlidingEstimator(clf, scoring='roc_auc')

In [ ]:
X.shape

In [ ]:
scores = cross_val_multiscore(time_decod, X, y, cv=cv)

In [ ]:
scores.shape

In [ ]:
plt.plot(epochs.times, scores.mean(axis=0));

In [ ]:
clf = make_pipeline(StandardScaler(),
                    LinearModel(LogisticRegression()))


In [ ]:
time_decod = SlidingEstimator(clf, scoring='roc_auc')

In [ ]:
time_decod.fit(X, y)

In [ ]:
coef = get_coef(time_decod, 'filters_', inverse_transform=True)

In [ ]:
coef.shape

In [ ]:
evoked_coef = mne.EvokedArray(coef,
                              epochs.info,
                              tmin=epochs.times[0])

In [ ]:
evoked_coef.plot_joint(); 

In [ ]:
from sklearn.naive_bayes import GaussianNB

In [ ]:
clf_nb = make_pipeline(StandardScaler(), GaussianNB())

In [ ]:
time_decod_nb = SlidingEstimator(clf_nb, scoring='roc_auc')

In [ ]:
scores_nb = cross_val_multiscore(time_decod_nb, X, y, cv=cv)

In [ ]:
plt.plot(epochs.times, scores.mean(axis=0), 'b', label='LR')
plt.plot(epochs.times, scores_nb.mean(axis=0), 'r', label='GNB');
plt.legend();
plt.grid();

In [ ]:
scores.mean()

In [ ]:
scores_nb.mean()

In [ ]:
plt.plot(epochs.times, scores.std(axis=0), 'b', label='LR')
plt.plot(epochs.times, scores_nb.std(axis=0), 'r', label='GNB')
plt.legend();
plt.grid();